## 测试分组回测（group_trader）

In [2]:
from billions.data_loader import *
from billions.back_trader import *

def test(name, peride):
    fname = 'data/' + name + '.csv'
    cname = 'data/close.csv'
    ftype = 'csv'

    data = loader_factor_close(fname, cname, ftype)
    gt = group_trader(data, number=5, name=name)
    gt.run(peride=peride)
    p = gt.ploter()
    p.plot()

x = list(map(test, ['betastd'], ['D']))


## 测试海龟

In [6]:
from billions.data_loader import *
from billions.back_trader import *

class turtle(trader):
    def __init__(self, data, total_amount=1000000, fluent=1, profolio=my_profolio, peride='D', name='turtle'):
        super().__init__(data, total_amount=total_amount, fluent=fluent,
                         profolio=profolio, peride=peride, name=name)
        self.codes = np.unique(np.array(list(data.index))[:, 1])

        self.data = data
        self.ret = self.caculate_return()

    def trade(self, date):
        for code in self.codes:
            today = self.data.loc[date, code]

            if today.close > today.up:
                self.profolio.buy(code, today.close*100)
            if today.close < today.down:
                self.profolio.sell_all([code])
                # self.profolio.sell(code, today.close*100)

    def caculate_return(self):
        ret = pd.DataFrame(
            (self.data['close'] - self.data['close'].shift())/self.data['close'].shift())

        ret.columns = ['return']

        return ret


In [7]:
data = load_price('data/600000.csv')
data['up'] = data['high'].shift().rolling(20).apply(np.max)
data['down'] = data['low'].shift().rolling(20).apply(np.min)
data['pre_close'] = data['close'].shift()
data = data[20:]

t = turtle(data)
t.run()
p = t.ploter()
p.plot()
